### Scrape wikipedia for Toronto city data

In [2]:
import urllib.request
import pandas as pd
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from bs4 import BeautifulSoup
import geocoder
import folium 
import requests
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)

In [4]:
soup = BeautifulSoup(page,'lxml')

In [5]:
#print(soup.prettify())

In [6]:
all_tables=soup.find_all("table")

In [7]:
my_table=soup.find('table', class_='wikitable sortable')

Extract data from table into empty lists

In [8]:
A=[]
B=[]
C=[]


for row in my_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

Create dataframe with respective columns

In [9]:
import pandas as pd
df=pd.DataFrame(A, columns=['Postcode'])
df['Borough']=B
df['Neighbourhood']=C


In [10]:
# remove '\n'

df['Neighbourhood'] = df['Neighbourhood'].str.replace("\n", "")
df['Borough'] = df['Borough'].str.replace("\n", "")


Ignore cells with a borough that is Not assigned.

In [11]:
df = df[df.Borough !='Not assigned']

More than one neighborhood can exist in one postal code area. <br>
These two rows will be combined into one row with the neighborhoods separated with a comma

In [12]:
df1 = df.groupby("Postcode").agg(lambda x:', '.join(set(x)))
df1.reset_index(inplace=True)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 


In [13]:
for i in range(len(df1)):
    if df1.Neighbourhood.iloc[i] == 'Not assigned':
        df1.Neighbourhood.iloc[i] = df1.Borough.iloc[i]

### Final Table

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [14]:
df1.shape

(103, 3)

# Part 2

We need to get the latitude and the longitude coordinates of each neighborhood.

Note: the geocoder does not work for certain postalcodes, even with iterative execution.
I will proceed with using CSV alternative

In [15]:
# Latitude= []
# Longitude= []


# for row in df1.itertuples(index=True, name='Pandas'):
#     address = getattr(row, "Borough")+ ', ' + getattr(row, "Postcode") + ', Toronto, Ontario'
#     geolocator = Nominatim(user_agent="to_explorer")
#     location = geolocator.geocode(address)
#     lat = location.latitude
#     long = location.longitude
#     print(long)
# #     Latitude.append(lat)
# #     Longitude.append(long)


# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#    g = geocoder.google('{}, Toronto, Ontario'.format('M1E'))
#    lat_lng_coords = g.latlng

# #latitude = lat_lng_coords[0]
# #longitude = lat_lng_coords[1]

In [16]:
geo = pd.read_csv('http://cocl.us/Geospatial_data', names= ['Postcode', 'Latitude','Longitude'])
geo = geo.drop([0],axis=0)

In [17]:
result = pd.merge(df1, geo, how='left', on=['Postcode'])

In [18]:
result.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8066863,-79.1943534
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.7845351,-79.1604971
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761


# Part 3

Explore and cluster the neighborhoods in Toronto.<br> You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data.
<br><br>
<b>make sure:</b>
<br>
-to add enough Markdown cells to explain what you decided to do and to report any observations you make.<br>
-to generate maps to visualize your neighborhoods and how they cluster together.

In [19]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [20]:
result.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.8066863,-79.1943534
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.7845351,-79.1604971
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",43.7635726,-79.1887115
3,M1G,Scarborough,Woburn,43.7709921,-79.2169174
4,M1H,Scarborough,Cedarbrae,43.773136,-79.2394761


In [21]:
# create map of Toronto using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

Simplify the above map and segment and cluster only Boroughs with the word 'Toronto'

In [22]:
result.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke', "Queen's Park"], dtype=object)

In [23]:
filters = (result.Borough == "East Toronto") | (result.Borough == "West Toronto") | (result.Borough == "Downtown Toronto")
to_data = result.loc[filters].reset_index(drop=True)

In [24]:
to_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.6763574,-79.2930312
1,M4K,East Toronto,"Riverdale, The Danforth West",43.6795571,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.6689985,-79.3155716
3,M4M,East Toronto,Studio District,43.6595255,-79.340923
4,M4W,Downtown Toronto,Rosedale,43.6795626,-79.3775294


In [25]:
address = 'Downtown Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Downtown Toronto are 43.6563221, -79.3809161.


In [26]:
# create map of Toronto using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(to_data['Latitude'], to_data['Longitude'], to_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to

In [27]:
CLIENT_ID = 'D1D3SAPTCL4HIOAPSCO00Q5H5NSBDKHCV2CARVUDIETSL1CX' # your Foursquare ID
CLIENT_SECRET = 'W0A5KO41JBLHXMVJGAIDCJWJ5DBJIE5D1BKFXGREN2LHZXNM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
to_venues = getNearbyVenues(names=to_data['Neighbourhood'],
                                   latitudes=to_data['Latitude'],
                                   longitudes=to_data['Longitude']
                                  )

The Beaches
Riverdale, The Danforth West
The Beaches West, India Bazaar
Studio District
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Toronto Islands, Union Station
Toronto Dominion Centre, Design Exchange
Victoria Hotel, Commerce Court
Harbord, University of Toronto
Kensington Market, Chinatown, Grange Park
Railway Lands, King and Spadina, Harbourfront West, Island airport, South Niagara, CN Tower, Bathurst Quay
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Parkdale Village, Brockton, Exhibition Place
High Park, The Junction South
Parkdale, Roncesvalles
Swansea, Runnymede
Queen's Park
Business Reply Mail Processing Centre 969 Eastern


In [30]:
to_venues.groupby('Neighbourhood').Venue.count()

Neighbourhood
Berczy Park                                                                                                    54
Business Reply Mail Processing Centre 969 Eastern                                                              18
Cabbagetown, St. James Town                                                                                    48
Central Bay Street                                                                                             84
Christie                                                                                                       18
Church and Wellesley                                                                                           82
Dovercourt Village, Dufferin                                                                                   17
First Canadian Place, Underground city                                                                        100
Harbord, University of Toronto                                            

### Analyze Each Neighborhood

In [31]:
print('There are {} uniques categories.'.format(len(to_venues['Venue Category'].unique())))

There are 219 uniques categories.


In [32]:
# one hot encoding
to_onehot = pd.get_dummies(to_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
to_onehot['Neighbourhood'] = to_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

to_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
to_onehot.shape

(1594, 220)

In [34]:
to_grouped = to_onehot.groupby('Neighbourhood').mean().reset_index()
to_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.018519,0.0,0.0,0.000000,0.0,0.000000
1,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.055556
2,"Cabbagetown, St. James Town",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011905,0.0,...,0.0,0.0,0.0,0.0,0.011905,0.0,0.0,0.011905,0.0,0.011905
4,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000


Analyze top five venues for each neighbourhood:

In [35]:
num_top_venues = 5

for hood in to_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = to_grouped[to_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.06
2        Beer Bar  0.04
3  Farmers Market  0.04
4            Café  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.11
1           Yoga Studio  0.06
2         Garden Center  0.06
3               Brewery  0.06
4  Fast Food Restaurant  0.06


----Cabbagetown, St. James Town----
               venue  freq
0        Coffee Shop  0.08
1         Restaurant  0.06
2             Bakery  0.06
3               Café  0.06
4  Convenience Store  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1  Italian Restaurant  0.05
2      Ice Cream Shop  0.05
3                Café  0.05
4      Sandwich Place  0.04


----Christie----
                venue  freq
0       Grocery Store  0.22
1                Café  0.17
2                Park  0.11
3         Gas Station  0.06
4  Italian Restaurant  0.06


----Church

Organize into a dataframe:

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = to_grouped['Neighbourhood']

for ind in np.arange(to_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(to_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Beer Bar,Seafood Restaurant,Farmers Market,Steakhouse,Café,Liquor Store
1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Park,Pizza Place,Burrito Place,Restaurant,Brewery,Smoke Shop
2,"Cabbagetown, St. James Town",Coffee Shop,Café,Bakery,Restaurant,Pizza Place,Italian Restaurant,Pub,Convenience Store,Beer Store,Caribbean Restaurant
3,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Sandwich Place,Juice Bar,Japanese Restaurant,Burger Joint,Bar,Spa
4,Christie,Grocery Store,Café,Park,Nightclub,Diner,Baby Store,Restaurant,Athletics & Sports,Italian Restaurant,Coffee Shop


In [37]:
to_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.018519,0.0,0.0,0.000000,0.0,0.000000
1,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.055556
2,"Cabbagetown, St. James Town",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000
3,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011905,0.0,...,0.0,0.0,0.0,0.0,0.011905,0.0,0.0,0.011905,0.0,0.011905
4,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000


### Cluster Neighbourhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [38]:
# set number of clusters
kclusters = 5

to_grouped_clustering = to_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(to_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 4, 0, 0, 1, 0,
       0, 0, 0, 0, 3, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [39]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

to_merged = to_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
to_merged = to_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

to_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.6763574,-79.2930312,3,Other Great Outdoors,Pub,Neighborhood,Trail,Health Food Store,Yoga Studio,Dim Sum Restaurant,Deli / Bodega,Department Store,Dessert Shop
1,M4K,East Toronto,"Riverdale, The Danforth West",43.6795571,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Caribbean Restaurant,Sports Bar,Pizza Place,Spa
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.6689985,-79.3155716,0,Park,Sandwich Place,Liquor Store,Pub,Steakhouse,Fish & Chips Shop,Sushi Restaurant,Ice Cream Shop,Pizza Place,Food & Drink Shop
3,M4M,East Toronto,Studio District,43.6595255,-79.340923,0,Café,Coffee Shop,Gastropub,Bakery,Brewery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place
4,M4W,Downtown Toronto,Rosedale,43.6795626,-79.3775294,1,Park,Trail,Playground,Yoga Studio,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


### Visualize the resulting clusters

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Latitude'], to_merged['Longitude'], to_merged['Neighbourhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster. 
<br>Based on the defining categories, you can then assign a name to each cluster.

In [41]:
to_merged.loc[to_merged['Cluster Labels'] == 0, to_merged.columns[[1] + list(range(5, to_merged.shape[1]))]].head()


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Caribbean Restaurant,Sports Bar,Pizza Place,Spa
2,East Toronto,0,Park,Sandwich Place,Liquor Store,Pub,Steakhouse,Fish & Chips Shop,Sushi Restaurant,Ice Cream Shop,Pizza Place,Food & Drink Shop
3,East Toronto,0,Café,Coffee Shop,Gastropub,Bakery,Brewery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place
5,Downtown Toronto,0,Coffee Shop,Café,Bakery,Restaurant,Pizza Place,Italian Restaurant,Pub,Convenience Store,Beer Store,Caribbean Restaurant
6,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Fast Food Restaurant,Men's Store,Mediterranean Restaurant,Hotel,Gym
